In [1]:
import pandas as pd
import numpy as np
import bitcoin
import datetime
from ping3 import ping
import random
import json
import re
import multiprocess as mp

from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

%pwd

'e:\\Codes\\PythonWorkspace\\CSE5005-Project\\data'

In [3]:
with open("./website.json", "r", encoding="utf8") as f:
    website_list = json.load(f)

website_list[:10]

[{'id': '2205',
  'name': '人人网',
  'logo': 'http://reg007.u.qiniudn.com/app/renren.jpg',
  'desc': '人人网，中国领先的实名制SNS社交网络。加入人人网，找到老同学，结识新朋友。',
  'category': '社交',
  'home': 'http://www.renren.com/'},
 {'id': '2217',
  'name': '途牛旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/tuniu.jpg',
  'desc': '途牛旅游网-中国专业全面的旅游网,客户满意度94%;提供自助游(自由行),国内旅游,出境旅游,自驾游,公司旅游。低价保证,专业服务,九大出游保障,服务超百万人出游。',
  'category': '旅行',
  'home': 'http://www.tuniu.com/'},
 {'id': '2224',
  'name': '酷讯旅游网',
  'logo': 'http://reg007.u.qiniudn.com/app/kuxun.jpg',
  'desc': '酷讯旅游网(Kuxun.cn)是中国领先的在线旅游媒体。为您提供低价可靠的国内国际机票，特价机票99元起，另提供全国20万家酒店预订查询、酒店评论，最低2折起，以及列车时刻表、50000多旅游景点的旅游攻略信息，为您的度假提供最佳的旅游线路。',
  'category': '旅行',
  'home': 'http://www.kuxun.cn/'},
 {'id': '2246',
  'name': '金山词霸',
  'logo': 'http://reg007.u.qiniudn.com/app/iciba.jpg',
  'desc': '爱词霸英语为广大网友提供在线翻译、在线词典、英语口语、英语学习资料、汉语词典，金山词霸下载等服务，致力于为您提供优质权威的在线英语服务，是5000万英语学习者的首选。',
  'category': '工具',
  'home': 'http://www.iciba.com/'},
 {'id': '2247',
  'name': '一听音乐网',
  '

In [9]:
pattern = re.compile("http://(.+?)/")
url_list = [re.findall(pattern, dic["home"])[0] for dic in website_list]

len(url_list)
url_list[:10]

212

['www.renren.com',
 'www.tuniu.com',
 'www.kuxun.cn',
 'www.iciba.com',
 'www.1ting.com',
 'www.youku.com',
 'www.zol.com.cn',
 'www.baihe.com',
 'www.izhenxin.com',
 'www.taobao.com']

In [5]:
url = random.choice(url_list)
delay = ping(url)

url
delay

'www.pchome.net'

0.031000137329101562

In [10]:
delete_url=[]
for url in url_list:
    if not ping(url):
        delete_url.append(url)
        
for url in delete_url:
    url_list.remove(url)

len(url_list)

173

In [11]:
def gen_addr():
    # 生成一个随机的密钥
    while True:
        # 生成一个用十六进制表示的长 256 位的私钥（str类型）
        private_key = bitcoin.random_key()
        # 解码为十进制的整形密钥
        decoded_private_key = bitcoin.decode_privkey(private_key, "hex")
        if 0 < decoded_private_key < bitcoin.N:
            break
    # print(private_key) #密钥（十六进制,长 256 位）

    # 计算地址
    # 用 WIF 格式编码密钥
    wif_encoded_private_key = bitcoin.encode_privkey(decoded_private_key, "wif")
    # 用 01 标识的压缩密钥
    compressed_private_key = private_key + "01"
    # 生成 WIF的压缩格式
    wif_compressed_private_key = bitcoin.encode_privkey(
        bitcoin.decode_privkey(compressed_private_key, "hex"), "wif"
    )
    # 计算公钥坐标 K = k * G
    public_key = bitcoin.fast_multiply(bitcoin.G, decoded_private_key)
    # 计算公钥
    hex_encoded_public_key = bitcoin.encode_pubkey(public_key, "hex")
    # 计算压缩公钥
    # if public_key[1] % 2 == 0:  # 两种方式均可
    if public_key[1] & 1 == 0:
        compressed_prefix = "02"
    else:
        compressed_prefix = "03"
    # 转十六也可用 bitcoin.encode(xxx, 16)
    hex_compressed_public_key = compressed_prefix + hex(public_key[0])[2:]
    # print(f'压缩公钥（十六进制）{hex_compressed_public_key} '
    #      '（02 开头代表 y 是偶数，03 开头代表 y 是奇数）')
    # 传入公钥坐标对象/十六进制公钥值，输出同样的地址
    address = bitcoin.pubkey_to_address(public_key)
    # print(address) #地址（b58check）（1 开头
    return address

In [12]:
df_user = pd.DataFrame(columns=["user_id", "address", "url"])
for i in range(2000):
    url = random.choice(url_list)
    # while not ping(url):
    #     url=random.choice(url_list)
    df_user.loc[i] = [i, gen_addr(), url]

df_user

,user_id,address,url
0,0,1GdNFUjMYGVJVue81eBMC4FVTLNuU5wSr9,www.chuanke.com
1,1,1FSRUMHyD3JyrTyvNgCVhaeknvFBCneMpa,www.ihou.com
2,2,16PU7kdUjukaujZQk2vN5gw3vmkbFJJ5RE,www.discuz.net
3,3,1DzWPykBF7Y9Xk7rGPWqGJrgzTjptUJujs,www.55bbs.com
4,4,1Q4mCqUi8gGSwf2njdsVGeBBrLgqr1XWzr,www.shopex.cn
...,...,...,...
1995,1995,1AUrebphGPQ8WfQ54KUwT4LZa73pK38dge,www.doc88.com
1996,1996,1CzbsnvcQCmoqfmfmjaL8oUyafCVrYGeim,www.loveq.cn
1997,1997,15PnrYJPqynKmuhqadXns86Kj3Mujc7jC3,www.120ask.com
1998,1998,1GgHj3fwejLghtKETvTkkSRSpj1nDVkMVE,www.gongchang.com


In [13]:
df_user.to_csv("user.csv", index=False)

---

In [19]:
!python gen_tx.py

In [3]:
df_tx = pd.read_pickle("./tx_49986.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-17 08:14:45.232276,2022-10-17 08:14:45.257685,0.025409,0,13ahFQNEenbTk1BwDknyYr5HXxfCT66pB2,1FgM5kyCtqgi7X3tgeYNePT6wFmY9XEp4J,2.095832
1,2022-10-17 08:14:45.257524,2022-10-17 08:14:45.293283,0.035759,0,17GP5qV9pqhTwaUKZFAkzAPodToxjKYjHA,14QPEv2FwYwQd6jvXtDFqtWa6WvaM3MNxZ,2.222274
2,2022-10-17 08:14:45.287651,2022-10-17 08:14:45.338839,0.051188,0,1Fs3JU4QtFq2F1uw5NMUBvhqbgDSyqdiFE,1BUNB8gE189GwCGHNETrTWfFgW6EnLMV71,0.859616
3,2022-10-17 08:14:45.313849,2022-10-17 08:14:45.339593,0.025744,0,18y5NJ4Yqx5T9tGFAXmFn9hJsFfmFUznbX,15vrPyAscByU4wxD8wEBfEgMvkLKSPWk36,4.911373
4,2022-10-17 08:14:45.363164,2022-10-17 08:14:45.372685,0.009521,0,1GxushZQxEGoFj3JbnDHf9d6UoRBwZCfju,1HeDYgvMbm5io1hghFGTJMTd6JrcC5tyE5,0.195828
...,...,...,...,...,...,...,...
49981,2022-10-17 08:58:53.997158,2022-10-17 08:58:54.016623,0.019465,0,1JDKAj2ppHeGHBcffS4yfsxNQEQ8X6soPU,14W2vwfhX68hxcJbGkaobsPjgjduGVkzTK,6.264613
49982,2022-10-17 08:58:54.053589,2022-10-17 08:58:54.077447,0.023858,0,18Tm43rHTQ4Q3oHsiuevGVK1u5ZQChRipo,19aCywvYu6jZ8kEEt44giTh59DQ1mCDA7M,7.999048
49983,2022-10-17 08:58:54.047145,2022-10-17 08:58:54.152313,0.105168,1,16xnPVkNerYQk4hoqFyaDxqyUHzkCKtXS1,1Hi714t4Pi9TtuBJXH4p8X4AM3wGCtTz1w,9.654536
49984,2022-10-17 08:58:54.079783,2022-10-17 08:58:54.180863,0.101080,0,1JiKyHda5H3cHsYtHAGaQK4wPkzmAD13Ah,1J2mycSNNia6Mf4NeXoASDfnrNpbmZoyD8,7.468506


In [4]:
sum(df_tx["label"] == 1)

12545

---

---

Generate training data

In [3]:
df_tx = pd.read_pickle("./tx_49986.pkl")

df_tx

,send_timestamp,recv_timestamp,latency,label,from,to,value
0,2022-10-17 08:14:45.232276,2022-10-17 08:14:45.257685,0.025409,0,13ahFQNEenbTk1BwDknyYr5HXxfCT66pB2,1FgM5kyCtqgi7X3tgeYNePT6wFmY9XEp4J,2.095832
1,2022-10-17 08:14:45.257524,2022-10-17 08:14:45.293283,0.035759,0,17GP5qV9pqhTwaUKZFAkzAPodToxjKYjHA,14QPEv2FwYwQd6jvXtDFqtWa6WvaM3MNxZ,2.222274
2,2022-10-17 08:14:45.287651,2022-10-17 08:14:45.338839,0.051188,0,1Fs3JU4QtFq2F1uw5NMUBvhqbgDSyqdiFE,1BUNB8gE189GwCGHNETrTWfFgW6EnLMV71,0.859616
3,2022-10-17 08:14:45.313849,2022-10-17 08:14:45.339593,0.025744,0,18y5NJ4Yqx5T9tGFAXmFn9hJsFfmFUznbX,15vrPyAscByU4wxD8wEBfEgMvkLKSPWk36,4.911373
4,2022-10-17 08:14:45.363164,2022-10-17 08:14:45.372685,0.009521,0,1GxushZQxEGoFj3JbnDHf9d6UoRBwZCfju,1HeDYgvMbm5io1hghFGTJMTd6JrcC5tyE5,0.195828
...,...,...,...,...,...,...,...
49981,2022-10-17 08:58:53.997158,2022-10-17 08:58:54.016623,0.019465,0,1JDKAj2ppHeGHBcffS4yfsxNQEQ8X6soPU,14W2vwfhX68hxcJbGkaobsPjgjduGVkzTK,6.264613
49982,2022-10-17 08:58:54.053589,2022-10-17 08:58:54.077447,0.023858,0,18Tm43rHTQ4Q3oHsiuevGVK1u5ZQChRipo,19aCywvYu6jZ8kEEt44giTh59DQ1mCDA7M,7.999048
49983,2022-10-17 08:58:54.047145,2022-10-17 08:58:54.152313,0.105168,1,16xnPVkNerYQk4hoqFyaDxqyUHzkCKtXS1,1Hi714t4Pi9TtuBJXH4p8X4AM3wGCtTz1w,9.654536
49984,2022-10-17 08:58:54.079783,2022-10-17 08:58:54.180863,0.101080,0,1JiKyHda5H3cHsYtHAGaQK4wPkzmAD13Ah,1J2mycSNNia6Mf4NeXoASDfnrNpbmZoyD8,7.468506


In [4]:
df_user=pd.read_csv("./user.csv")

user_list=df_user["address"].values.tolist()

user_list[:5]

['1LUjEfWLZhrndJBkkqXABxpaQNQskBZGqY',
 '17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m',
 '1DVBvtVz8Gj61Sjvea61TP3vD46xx7ApgB',
 '1Gc2G3fHt41uFWbmfLSwLuevXx72m36HGB',
 '1JKifazQtMr4F4hBQhBF4Pc2M6cPRkXEn7']

In [17]:
user_list.index("17Bo6VWMnfks9Amz35MKzKsBZqzBdU7P8m")

1

In [5]:
def get_user_id(row):
    from_user_id=user_list.index(row["from"])
    to_user_id=user_list.index(row["to"])
    
    row["from_user_id"]=from_user_id
    row["to_user_id"]=to_user_id
    
    return row

In [6]:
df_data=df_tx.copy()
df_data=df_data.apply(get_user_id, axis=1)

df_data

,send_timestamp,recv_timestamp,latency,label,from,to,value,from_user_id,to_user_id
0,2022-10-17 08:14:45.232276,2022-10-17 08:14:45.257685,0.025409,0,13ahFQNEenbTk1BwDknyYr5HXxfCT66pB2,1FgM5kyCtqgi7X3tgeYNePT6wFmY9XEp4J,2.095832,605,1583
1,2022-10-17 08:14:45.257524,2022-10-17 08:14:45.293283,0.035759,0,17GP5qV9pqhTwaUKZFAkzAPodToxjKYjHA,14QPEv2FwYwQd6jvXtDFqtWa6WvaM3MNxZ,2.222274,589,1242
2,2022-10-17 08:14:45.287651,2022-10-17 08:14:45.338839,0.051188,0,1Fs3JU4QtFq2F1uw5NMUBvhqbgDSyqdiFE,1BUNB8gE189GwCGHNETrTWfFgW6EnLMV71,0.859616,1501,253
3,2022-10-17 08:14:45.313849,2022-10-17 08:14:45.339593,0.025744,0,18y5NJ4Yqx5T9tGFAXmFn9hJsFfmFUznbX,15vrPyAscByU4wxD8wEBfEgMvkLKSPWk36,4.911373,550,393
4,2022-10-17 08:14:45.363164,2022-10-17 08:14:45.372685,0.009521,0,1GxushZQxEGoFj3JbnDHf9d6UoRBwZCfju,1HeDYgvMbm5io1hghFGTJMTd6JrcC5tyE5,0.195828,1750,1754
...,...,...,...,...,...,...,...,...,...
49981,2022-10-17 08:58:53.997158,2022-10-17 08:58:54.016623,0.019465,0,1JDKAj2ppHeGHBcffS4yfsxNQEQ8X6soPU,14W2vwfhX68hxcJbGkaobsPjgjduGVkzTK,6.264613,755,1057
49982,2022-10-17 08:58:54.053589,2022-10-17 08:58:54.077447,0.023858,0,18Tm43rHTQ4Q3oHsiuevGVK1u5ZQChRipo,19aCywvYu6jZ8kEEt44giTh59DQ1mCDA7M,7.999048,846,692
49983,2022-10-17 08:58:54.047145,2022-10-17 08:58:54.152313,0.105168,1,16xnPVkNerYQk4hoqFyaDxqyUHzkCKtXS1,1Hi714t4Pi9TtuBJXH4p8X4AM3wGCtTz1w,9.654536,1220,1741
49984,2022-10-17 08:58:54.079783,2022-10-17 08:58:54.180863,0.101080,0,1JiKyHda5H3cHsYtHAGaQK4wPkzmAD13Ah,1J2mycSNNia6Mf4NeXoASDfnrNpbmZoyD8,7.468506,1599,1669


In [7]:
df_data.drop(columns=["send_timestamp", "from", "to"], inplace=True)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
0,2022-10-17 08:14:45.257685,0.025409,0,2.095832,605,1583
1,2022-10-17 08:14:45.293283,0.035759,0,2.222274,589,1242
2,2022-10-17 08:14:45.338839,0.051188,0,0.859616,1501,253
3,2022-10-17 08:14:45.339593,0.025744,0,4.911373,550,393
4,2022-10-17 08:14:45.372685,0.009521,0,0.195828,1750,1754
...,...,...,...,...,...,...
49981,2022-10-17 08:58:54.016623,0.019465,0,6.264613,755,1057
49982,2022-10-17 08:58:54.077447,0.023858,0,7.999048,846,692
49983,2022-10-17 08:58:54.152313,0.105168,1,9.654536,1220,1741
49984,2022-10-17 08:58:54.180863,0.101080,0,7.468506,1599,1669


In [8]:
df_data["recv_timestamp"]=df_data["recv_timestamp"].apply(lambda x: x.value)

df_data

,recv_timestamp,latency,label,value,from_user_id,to_user_id
0,1665994485257685000,0.025409,0,2.095832,605,1583
1,1665994485293283000,0.035759,0,2.222274,589,1242
2,1665994485338839000,0.051188,0,0.859616,1501,253
3,1665994485339593000,0.025744,0,4.911373,550,393
4,1665994485372685000,0.009521,0,0.195828,1750,1754
...,...,...,...,...,...,...
49981,1665997134016623000,0.019465,0,6.264613,755,1057
49982,1665997134077447000,0.023858,0,7.999048,846,692
49983,1665997134152313000,0.105168,1,9.654536,1220,1741
49984,1665997134180863000,0.101080,0,7.468506,1599,1669


In [9]:
df_data.to_pickle(f"./data_{len(df_data)}.pkl")

In [14]:
df_data[["recv_timestamp", "from_user_id", "to_user_id", "value", "label"]].values[:5]

array([[1.66599449e+18, 6.05000000e+02, 1.58300000e+03, 2.09583233e+00,
        0.00000000e+00],
       [1.66599449e+18, 5.89000000e+02, 1.24200000e+03, 2.22227408e+00,
        0.00000000e+00],
       [1.66599449e+18, 1.50100000e+03, 2.53000000e+02, 8.59616216e-01,
        0.00000000e+00],
       [1.66599449e+18, 5.50000000e+02, 3.93000000e+02, 4.91137335e+00,
        0.00000000e+00],
       [1.66599449e+18, 1.75000000e+03, 1.75400000e+03, 1.95827537e-01,
        0.00000000e+00]])